# Panel Data

Sometimes, data comes in such a way that many observations share certain common features. For example, several measurements can be made in the same location, under the same condition, or for the same subject. To understand the data and extract meaningful insights, we often need to aggregate these observations. This is where the groupby() function comes into play.

## Loading

As always, let's start by importing pandas and loading and cleaning our dataset.

In [1]:
import pandas as pd

# Load the data
df = pd.read_csv("data/sp500_q1_2025.csv")

# Look at dates - back in the ISO format!
print("Look at our dates:\n", df.DlyCalDt.head())

# Convert the 'datadate' column to a datetime object
df.DlyCalDt = pd.to_datetime(df.DlyCalDt)

FileNotFoundError: [Errno 2] No such file or directory: 'data/sp500_q1_2025.csv'

We'll stop short of setting the index as our datetime value though. This is because an index must have unique values, and because this panel data contains lots of different company stocks for just one quarter of a year, we'll see the same date lots of times.

In [ ]:
# How many unique dates exist in the data frame
print("Number of unique dates", df.DlyCalDt.nunique())

# Last date in the dataset
print("Last date", df.DlyCalDt.max())

# First date in the dataset
print("First date", df.DlyCalDt.min())

## Cleaning

Let's not forget data cleaning! Do we have missing data? Where?

In [ ]:
# Check for missing values
print("Missing data", df.isnull().sum().sum())

# Find missing data
df[df.isnull().any(axis=1)]

# Given the amount of missing data, it may be best to drop these rows
df.dropna(inplace=True)

print("Missing data after cleaning", df.isnull().sum().sum())

## Exploring

Let's explore this panel data a bit more, to answer some questions:

- How many tickers are considered
- How many securities are considered
- How many companies are considered
- Which exchanges are considered
- Which exchanges appear most


In [ ]:
print("Unique tickers:", df.Ticker.nunique())
print("Unique companies", df.PERMCO.nunique())
print("Unique securities", df.PERMNO.nunique())
# Notice the discrepancy between these values - we'll look more at why this is when we learn to group

# If we use unique() instead of nunique() we'll get the actual values
print("Unique exhanges:", df.PrimaryExch.unique())
print("Exchanges by appearance:", df.PrimaryExch.value_counts())

## Grouping

What if we wanted to calculate daily returns in this data set. Is it as simple as using `pct_change()`? Let's try.

In [ ]:
df["Returns"] = df.DlyClose.pct_change()
df # switch to 25 per page and go to page 3 (row index 60) to see the problem!

Can you see what's gone wrong here? Our first calculated daily return for American Airlines is using Agilent's last closing price. This hopefully gets across the importance of *grouping*, particularly useful with this kind of panel data.


We can solve this with the `groupby()` method of data frames.

In [ ]:
df["Returns"] = df.groupby("PERMNO").DlyClose.pct_change()
df.head(65) # just navigate to the last page of the dataframe view for row index 60

Perfect! Grouping is a very powerful way to manipulate panel data. Once you've grouped, you can call functions and they will be applied groupwise as we saw above. Here are some other common functions with groups:

In [ ]:
# Identify the number of rows in each group
print("Number of rows per group", df.groupby("Ticker").size())

# Subset a specific group
apple = df.groupby("Ticker").get_group("AAPL")
apple


Let's see what else we can do with grouping. Recall that we had more tickers than companies. Let's see why that is by looking at how many unique tickers belong to each company (using `Ticker` and `PERMCO`). Then let's list those companies.

In [ ]:
# First we create a series with the number of unique stocks for each company
ticker_counts = df.groupby("PERMCO").Ticker.nunique()
multi_permco = ticker_counts[ticker_counts > 1].index

# Then we index that series with a boolean expression
df[df.PERMCO.isin(multi_permco)]

### Exercise: Tick Tick

**Part 1** Identify the number of unique tickers traded on each exchange.

In [ ]:
## YOUR CODE GOES HERE

**Part 2** Then identify any securities that share a ticker.

In [ ]:
## YOUR CODE GOES HERE

## Aggregation

Aggregation functions like `mean()`, `median()`, `sum()`, `min()`, `max()`, `first()`, `last()` and `std()` can be applied to grouped data to give insights across panel data. Say we wanted the average daily return of each traded security, or the max volume traded on any given day for each security?

The exercises above helped us identify that the `PERMNO` column corresponds to unique securities, so let's use that for grouping from now on. 

In [ ]:
df.groupby("PERMNO").Returns.mean()

Useful, but only to a point. The `PERMNO` value is just a number to most of us. What if we want a ticker or name for the security? Let's look at grouping by multiple columns to help!

In [ ]:
df.groupby(["PERMNO", "Ticker"], as_index=False).DlyClose.min() # as_index is optional. It keeps our columns as columns, allowing pretty display of a dataframe

df.groupby(["PERMNO", "SecurityNm"], as_index=False).DlyClose.first() # notice this is different to min(), and the first price of the period

df.groupby(["PERMNO", "Ticker"], as_index=False).SecurityNm.first() # first() is commonly used for aggregating like, non-numeric data


Once we've done these sorts of aggregation, we're often curious to see who sits at the top or the bottom of the distribution. We can use `nlargest()` and its antonym here. Note that `as_index=False` doesn't work here easily, since these functions refer to the index!

In [ ]:
df.groupby("SecurityNm").Returns.mean().nsmallest()

df.groupby("SecurityNm").DlyVol.max().nlargest()


We can also group by multiple columns! This can be helpful when doing aggregation, for example, to find high performers in each month. Because our date is just a regular column, we need to specify `.dt` to use any datetime functions.

In [ ]:
# First create a column to specify the month
df["Month"] = df.DlyCalDt.dt.month_name()

# Then use it to group and aggregate for max closing price each month
df.groupby(["Ticker", "Month"], as_index=False).DlyClose.max()

### Exercise: Good Days

Which two days of the week see the highest average close in this data set, and what is the average close for those days?  

In [ ]:
## YOUR CODE GOES HERE

### Exercise: Trading Exchanges

Next identify the total trading volume of each exchange.

In [ ]:
## YOUR CODE GOES HERE

### Exercise: The 1000 Club

For securities that reached a closing price above 1000, how many times in each month, did they acheive this?

In [ ]:
## YOUR CODE GOES HERE

## Multiple Aggregation

We can use the `agg()` method, and pass it a dictionary to do multiple aggregations at once on grouped data. This can be helpful for further analyses, or for producing a more descriptive aggregated data frame.

In [ ]:
df.groupby("PERMNO").agg({"SecurityNm": "first", "DlyClose": "mean"})

df.groupby("PERMNO").agg({"SecurityNm": "first", "DlyClose": ["first", "last"], "DlyVol": "sum"}).nlargest(5, columns=("DlyVol", "sum"))

### Quick Quarter Query

Using multiple aggregation, create an aggregated data frame with ticker and security name, the first open price in the period for each security and the last close price in the period for each security. Create a new column in this aggregated data frame that shows the price difference between final close and initial open for each security.

In [ ]:
## YOUR CODE GOES HERE